In [39]:
import sys
import requests
from bs4 import BeautifulSoup
import random
import os
import yaml
import csv
import pandas as pd
import urllib
import urllib.request
import time

In [2]:
with open("../config.yaml", "r") as f:
    config = yaml.safe_load(f)


In [99]:
class BigbasketScrapper:
    
    def __init__(self, soup):
        self.soup = soup
        self.product_list = list()
        
    def get_images(self):
        html_list = self.soup.findAll('script')[4].text.replace('\\', '')[self.soup.findAll('script')[4].text.replace('\\', '').index('{'):-2].split(',')
        
        i=0

        while i < len(html_list):
            if 'mimage' in html_list[i]:
                image_link = html_list[i].split('"')[3].strip()
                i += 1
                while True:
                    if 'desc' in html_list[i]:
                        product_name = html_list[i].split('"')[3].strip()
                    elif 'w2' in html_list[i]:
                        qty = html_list[i].split('"')[3].strip()
                    elif 'brandName' in html_list[i]:
                        brandname = html_list[i].split('"')[3].strip()
                        break
                    i += 1
            
                self.product_list.append((brandname, product_name, qty, image_link))
                
                tempCheck = image_link.split('.')
                check = str(tempCheck[len(tempCheck) -1])
                ImageType = ".jpeg"
                
                if (check == "jpg" or check == "jpeg" or check == "png"):
                    
                    if check == "jpg" or check == "jpeg":
                        ImageType = ".jpeg"
                    else:
                        ImageType = ".png"
                        
                    if not os.path.exists(config['path_params']['output_images_path']  + '/' + brandname + '/' + product_name):
                        os.makedirs(config['path_params']['output_images_path'] + '/' + brandname + '/' + product_name)
                    
                    time.sleep(random.randrange(2, 10))
                    imagefile = open(config['path_params']['output_images_path']  + '/' + brandname + '/' + product_name + ImageType, 'wb')
                    imagefile.write(urllib.request.urlopen(image_link).read())
                    imagefile.close()
                i +=1
            else:
                i += 1

In [38]:
class MakeSoup:
    
    def __init__(self, url):
        self.url = url
        
    def GET_UA():
        #'data/user_agent_list/user_agent_list.txt' 
        with open(config['path_params']['user_agent_path'] + config['files_names']['user_agent_file']) as f:
            user_agents_list = f.readlines()
            
        return random.choice(user_agents_list)
        
    def makeSoup_url(self):
        headers = {'User-Agent': GET_UA()[:-1]}
        r = requests.get(self.url, headers=headers)
        soup = BeautifulSoup(r.content, 'html.parser')
        return soup

In [35]:
class BigbasketCrawler:
    
    def __init__(self):
        #'data/url_files/bigbasket.csv' 
        self.bigbasket_main_urls = pd.read_csv('../' + config['path_params']['url_files_path'] + config['files_names']['bigbasket_main_url_file'], sep ='|')
        self.bigbasket_scrapped_urls = list()
    
    def get_unscrapped_urls(self):
        if os.path.exists('../' + config['path_params']['url_files_path'] + config['files_names']['bigbasket_scrapped_urls']):
            df3 = self.bigbasket_main_urls.merge(pd.read_csv('../' + config['path_params']['url_files_path'] + config['files_names']['bigbasket_scrapped_urls'], sep = '|'), on='link')
            self.bigbasket_main_urls = self.bigbasket_main_urls[~self.bigbasket_main_urls['link'].isin(df3['link'])]
        
        return self.bigbasket_main_urls
    
    def get_soup(self):
        url_list = self.get_unscrapped_urls()
        url_list = url_list[~url_list['category'].isin(['Fresh Vegetables', 'Fresh Fruits', 'Organic Fruits & Vegetables'])].values
        
        for sub_category in url_list:
            print(sub_category)
            soup = MakeSoup(sub_category[2])
            scrapper = BigbasketScrapper(soup.makeSoup_url())
            scrapper.get_images()
            self.bigbasket_scrapped_urls.append(sub_category)
            pd.DataFrame(self.bigbasket_scrapped_urls, columns=['category', 'sub_category', 'link']).to_csv(config['path_params']['url_files_path'] + config['files_names']['bigbasket_scrapped_urls'], sep = '|', index= False)
        

In [36]:
bigbasket_crawler = BigbasketCrawler()

In [ ]:
bigbasket_crawler.get_soup()

In [45]:
headers = {'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0.1; Nexus 5X Build/MMB29P) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.5307.0 Mobile Safari/537.36 (compatible; Googlebot/2.1;  http://www.google.com/bot.html)'}
url = 'https://www.bigbasket.com/pc/foodgrains-oil-masala/masalas-spices/powdered-spices/?nc=cs'
r = requests.get(url, headers=headers)
soup = BeautifulSoup(r.content, 'html.parser')

In [47]:
soup.findAll(class_ = 'ng-isolate-scope')

[]

In [84]:
html_list = soup.findAll('script')[4].text.replace('\\', '')[soup.findAll('script')[4].text.replace('\\', '').index('{'):-2].split(',')

In [ ]:
i=0

product_list = list()

while i < len(html_list):
    if 'mimage' in html_list[i]:
        image_link = html_list[i].split('"')[3].strip()
        i += 1
        while True:
            if 'desc' in html_list[i]:
                product_name = html_list[i].split('"')[3].strip()
            elif 'w2' in html_list[i]:
                qty = html_list[i].split('"')[3].strip()
            elif 'brandName' in html_list[i]:
                brandname = html_list[i].split('"')[3].strip()
                break
            i += 1
            
        product_list.append((brandname, product_name, qty, image_link))
        i +=1
    else:
        i += 1

In [93]:
37706 - 37804


-98

In [88]:
'mimage' in '"mimage":"https://www.bigbasket.com/media/uploads/p/m/40077185_9-bb-royal-chillimirchi-powder.jpg"'

True

In [ ]:
html_list[5747:5852]